In [14]:
using DIVAnd
using PyCall
using CSV
using DataFrames
using Pkg
using NetCDF
crs = pyimport("cartopy")
ccrs = pyimport("cartopy.crs");
plt = pyimport("matplotlib.pyplot");

In [2]:
data = DataFrame(CSV.File("glodap_with_ttd_ages.csv"));

In [65]:
masklon = ncread("/home/ashao/data/lsmask.19294.nc","lon")
masklat = ncread("/home/ashao/data/lsmask.19294.nc","lat");
lsmask = ncread("/home/ashao/data/lsmask.19294.nc","lsmask");
lonidx = (masklon .> 110) .& (masklon .< 280)
latidx = (masklat .> -10) .& (masklat .< 70)
sigmas = range(26.1,27,step=0.1)
ocean_mask = repeat(reverse(lsmask[lonidx,latidx,:] .== 0,dims=2),outer=(1,1,size(sigmas,1)))
xi,yi,zi = ndgrid(masklon[lonidx],reverse(masklat[latidx]),sigmas);

In [69]:
subdata = dropmissing(data,:cfc11_age)# sigma = 26.2
# subdata = subdata[ (subdata.sigma0 .> 26.1) .& (subdata.sigma0 .< 26.3),: ]
x = (mod1.(subdata.longitude,360),subdata.latitude,subdata.sigma0)
grid_intp = (xi,yi,zi)
cfc11_values = subdata.cfc11_age;
grid_factors = (
    ones(size(xi))./1.875,
    ones(size(yi))./1.9,
    ones(size(zi))./0.025
    )
    
cfc11,s = DIVAndrun(ocean_mask,grid_factors,grid_intp,x,cfc11_values,(10.,5.,0.05),20.);
cfc11[cfc11 .< 0] .= 0.
plt.figure()
ax = plt.subplot(1,1,1,projection=ccrs.PlateCarree(central_longitude=390/2))
cax = ax.pcolormesh(xi[:,:,1],yi[:,:,1],cfc11[:,:,2],transform=ccrs.PlateCarree(),cmap=plt.cm.RdBu_r,vmin=0,vmax=120)
ax.scatter(subdata.longitude,subdata.latitude,s=2.,c="w",transform=ccrs.PlateCarree())
ax.contour(xi[:,:,1],yi[:,:,1],cfc11[:,:,1],levels = [5,10,20,40,80,120],transform=ccrs.PlateCarree(),colors="black")
ax.set_extent([110,280,-10,70],crs=ccrs.PlateCarree())
# ax.add_feature(crs.feature.COASTLINE, zorder=0, edgecolor="black")
plt.colorbar(cax,label = "CFC-11 Age (yr)",orientation="horizontal")
# plt.show()

PyObject <matplotlib.colorbar.Colorbar object at 0x7f2ecbbdf588>

In [67]:
o2data = data[(data.oxygenqc .== 1) .& (data.oxygen.>0),:]
x = (mod1.(o2data.longitude,360),o2data.latitude,o2data.sigma0)
o2_values = o2data.oxygen
o2,s = DIVAndrun(ocean_mask,grid_factors,grid_intp,x,o2_values,(10.,5.,0.05),10.);
o2[o2.<0] .= 0.

plt.figure()
ax = plt.subplot(1,1,1,projection=ccrs.PlateCarree(central_longitude=390/2))
cax = ax.pcolormesh(xi[:,:,1],yi[:,:,1],o2[:,:,1],transform=ccrs.PlateCarree(),cmap=plt.cm.RdBu_r)
ax.scatter(o2data.longitude,o2data.latitude,s=2.,c="w",transform=ccrs.PlateCarree())
ax.contour(xi[:,:,1],yi[:,:,1],o2[:,:,1],levels=range(0,stop=500,step=50),transform=ccrs.PlateCarree(),colors="black")
ax.set_extent([110,280,-10,70],crs=ccrs.PlateCarree())
# ax.gridlines()
# ax.coastlines()
plt.colorbar(cax,label = "Oxygen [umol kg-1]",orientation="horizontal")
# plt.show()

PyObject <matplotlib.colorbar.Colorbar object at 0x7f2ecbf56940>

In [70]:
plt.show()

In [72]:
o2atts = Dict(
    "units" => "umol kg-1",
    "long_name" => "concentration of oxygen in seawater")
ageatts = Dict(
    "units" => "years",
    "long_name" => "age of seawater inferred from 1d TTD method using pCFC-11"
    )
sigmaatts = Dict(
    "units" => "kg m-3",
    "long_name" => "potential density anomaly referenced to 0 dbar"
    )
fname = "gridded_oxygen_age.nc"
nccreate(fname,"o2","lon",masklon[lonidx],"lat",reverse(masklat[latidx]),"sigma0",sigmas,sigmaatts,atts=o2atts)
nccreate(fname,"cfc11_age","lon",masklon[lonidx],"lat",reverse(masklat[latidx]),"sigma0",sigmas,sigmaatts,atts=ageatts)
ncwrite(o2,fname,"o2")
ncwrite(cfc11,fname,"cfc11_age")